## Selection of LLM (`Local` or `AWS`)

In [1]:
selected_mode = input("Enter the mode (local or aws) :").lower()
print(f"{selected_mode = }")

selected_mode = 'aws'


## `LLM` Model Selection

In [2]:
import boto3
import ipywidgets as widgets
from IPython.display import display

if selected_mode == "aws":
    # Create a client for Bedrock
    bedrock_client = boto3.client('bedrock')

    # Get the model ids
    response = bedrock_client.list_foundation_models()

    # Extract model IDs
    model_ids_list = [model['modelId'] for model in response.get('modelSummaries', [])]

    # Create a dropdown widget
    model_dropdown = widgets.Dropdown(
        options=model_ids_list,
        description='Choose a Model:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Function to capture the selected model
    selected_model = None
    def on_model_change(change):
        global selected_model
        selected_model = change['new']
        print(f"Selected Model ID: {selected_model}")

    # Attach the callback function
    model_dropdown.observe(on_model_change, names='value')

    # Display the dropdown
    display(model_dropdown)


Dropdown(description='Choose a Model:', layout=Layout(width='50%'), options=('amazon.titan-tg1-large', 'amazon…

Selected Model ID: amazon.nova-pro-v1:0


In [3]:
# Check the selected model and if contains "nova" then rename the model with a prefix "us."     
if selected_mode == "aws":
    if "nova" in selected_model and "us" not in selected_model:
        selected_model = "us." + selected_model

    print(f"Selected Model ID: {selected_model}")
else:
    print(f"Selected Model ID (Local): Ollama/llama3.2")

Selected Model ID: us.amazon.nova-pro-v1:0


## Setting up an Agent with `CrewAI` (without **tools**)

### `Step 1`: Setting the LLM 

### Pre-requisites
If you are using AWS, you need to install the `boto3` library.
If you are using local, you need to install the `ollama` library.






In [5]:
from crewai import LLM

# Check for the mode and set the LLM accordingly    
if selected_mode == 'local':
    llm = LLM(model="ollama/llama3.2",
              base_url="http://localhost:11434" )
else:
    llm = LLM(model=selected_model)
    
print(f"CrewAI LLM to be used: {llm.model}")

CrewAI LLM to be used: us.amazon.nova-pro-v1:0


### `Step 2`: Create an Agent

In [6]:
from crewai import Agent

# Senior Technical Writer
senior_technical_writer = Agent(

    role="Senior Technical Writer",
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    backstory="""You are an experienced technical writer
                 with expertise in simplifying complex
                 concepts, structuring content for readability,
                 and ensuring accuracy in documentation.""",
    llm=llm,       
    verbose=True
)

# Senior Research Analyst
research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                 in gathering accurate data, identifying key trends, 
                 and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

# Code reviwer Agent
code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                 experience in writing, reviewing, and optimizing 
                 production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

# Legal Advisor Agent
legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                 of contract law, regulatory frameworks, 
                 and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


### `Step 3`: Define Tasks 

In [7]:
from crewai import Task

# Task 1: Task for Senior Technical Writer
writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)

### `Step 4`: Create a Crew 

In [8]:
from crewai import Crew

crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

### `Step 5`: Run the Crew 

In [9]:
# Run the crew
result = crew.kickoff(inputs={"topic": "Vision Transformers"})

# Agent: Senior Technical Writer
## Task: Write a well-structured, engaging,
                   and technically accurate article
                   on Vision Transformers.


# Agent: Senior Technical Writer
## Final Answer: 
---

**Understanding Vision Transformers: Revolutionizing Computer Vision**

In the ever-evolving field of artificial intelligence, Vision Transformers (ViTs) have emerged as a groundbreaking approach to computer vision tasks. Traditionally, Convolutional Neural Networks (CNNs) have dominated the landscape of image recognition and classification. However, Vision Transformers are challenging this status quo by offering a new paradigm that leverages the strengths of transformer models, originally designed for natural language processing.

### What are Vision Transformers?

Vision Transformers adapt the transformer architecture, initially introduced in the context of sequence-to-sequence tasks like machine translation, to handle visual data. Unlike CNNs, which process

In [10]:
from IPython.display import Markdown

Markdown(result.raw)

---

**Understanding Vision Transformers: Revolutionizing Computer Vision**

In the ever-evolving field of artificial intelligence, Vision Transformers (ViTs) have emerged as a groundbreaking approach to computer vision tasks. Traditionally, Convolutional Neural Networks (CNNs) have dominated the landscape of image recognition and classification. However, Vision Transformers are challenging this status quo by offering a new paradigm that leverages the strengths of transformer models, originally designed for natural language processing.

### What are Vision Transformers?

Vision Transformers adapt the transformer architecture, initially introduced in the context of sequence-to-sequence tasks like machine translation, to handle visual data. Unlike CNNs, which process images through a series of convolutional filters, Vision Transformers treat an image as a sequence of patches. Each patch is then linearly embedded and processed similarly to how words are processed in a sentence.

### How Do Vision Transformers Work?

1. **Image Patching**:
    An input image is divided into fixed-size patches. For instance, a 224x224 image might be split into 16x16 patches, resulting in 196 patches of 16x16 pixels each.

2. **Linear Embedding**:
    Each patch is flattened into a vector and then linearly embedded to a desired dimension. Additionally, a learnable position embedding is added to retain spatial information.

3. **Transformer Encoder**:
    The embedded patches, along with a special `[CLS]` token (similar to NLP tasks), are fed into a standard transformer encoder. The encoder consists of multiple layers of self-attention mechanisms and feed-forward neural networks.

4. **Classification**:
    The `[CLS]` token’s final representation is used for classification tasks. For other tasks like object detection or segmentation, different strategies may be employed.

### Advantages of Vision Transformers

1. **Scalability**:
    Vision Transformers benefit from the scalability of transformer models. They can be trained on massive datasets, leading to improved performance as the amount of data increases.

2. **Global Context**:
    Unlike CNNs, which have a limited receptive field, Vision Transformers can capture global context from the beginning, potentially leading to better understanding of complex scenes.

3. **Flexibility**:
    The architecture is highly flexible and can be adapted to various vision tasks beyond simple classification, including semantic segmentation and object detection.

### Challenges and Considerations

1. **Data Requirements**:
    Vision Transformers typically require significantly more data compared to CNNs to achieve optimal performance. This can be a limitation in scenarios where data is scarce.

2. **Computational Cost**:
    Training Vision Transformers is computationally expensive due to the large number of parameters and the quadratic complexity of self-attention mechanisms with respect to sequence length.

3. **Hybrid Approaches**:
    To mitigate some of these challenges, hybrid models that combine CNNs and Vision Transformers have been proposed. These models leverage the strengths of both architectures.

### Real-World Applications

Vision Transformers are being applied in various real-world scenarios, including:

- **Medical Imaging**: Enhancing the accuracy of disease detection in radiology.
- **Autonomous Vehicles**: Improving object recognition and scene understanding for safer navigation.
- **Retail**: Enabling advanced visual search and recommendation systems.

### Conclusion

Vision Transformers represent a significant shift in the approach to computer vision. By adapting the successful transformer architecture to visual data, they offer new possibilities for innovation and improvement in image-related tasks. As research continues to advance, we can expect Vision Transformers to play an increasingly important role in the future of artificial intelligence.

---

This article provides a comprehensive overview of Vision Transformers, detailing their mechanism, advantages, challenges, and real-world applications.

## Setting up an Agent with `CrewAI` (with **tools**)

### `Step 1`: Define a tool to perform a web search and add to the Agent and Task


In [11]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

# Define the tool
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)


# Defining the Agent
travel_agent = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations matching user preferences',
    backstory="You are an experienced travel agent specializing in personalized travel recommendations.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

# Defining the Task
task = Task(
    description="Based on the user's travel preferences: {preferences}, research and recommend suitable travel destinations.",
    expected_output="A list of recommended destinations with brief descriptions.",
    agent=travel_agent
)


### `Step 2`: Create and run the Crew 


In [12]:
# Create the Crew
crew = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
)


# User input for travel preferences
user_input = {
    "preferences": "I want a tropical beach vacation with great snorkeling and vibrant nightlife."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

2025-02-12 15:29:38,944 - 8232306752 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To find suitable tropical beach destinations with great snorkeling and vibrant nightlife, I should perform a web search for popular locations that match these criteria.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach destinations with great snorkeling and vibrant nightlife\"}"
## Tool Output: 
Must-see: Anse Lazio beach on Praslin Island and giant Aldabra tortoises on Curieuse Island. Phuket, Thailand: A Tropical Party Hub. Phuket offers the best of both worlds — serene beaches like Kata Noi and lively nightlife at Patong. Add in Thai cuisine and affordable luxury, and you've got an unbeatable tropical escape. 5. Phuket, Thailand: Unwind on beautiful beaches, dive into liv

In [13]:
# Display the result
Markdown(result.raw)

Here are some recommended tropical beach destinations with great snorkeling and vibrant nightlife:

1. **Phuket, Thailand**
   - *Description*: Phuket is Thailand's largest island, renowned for its stunning beaches, lively nightlife, and rich culture. It offers serene beaches like Kata Noi for relaxation and vibrant nightlife at Patong. Don’t miss the Phi Phi Islands, the Big Buddha, and Wat Chalong.

2. **Boracay, Philippines**
   - *Description*: Boracay is famous for its white sand beaches and clear blue waters, making it ideal for snorkeling. The island offers a range of accommodations suitable for both party-goers and families, with vibrant nightlife and various water activities.

3. **The Bahamas**
   - *Description*: The Bahamas is an enchanting archipelago with over 700 islands and cays, each offering its own slice of paradise. Nassau, the capital, provides vibrant nightlife, rich history, and cultural experiences. Explore colorful architecture and local culture at the Pompey Museum.

4. **Praslin Island, Seychelles**
   - *Description*: Praslin Island is home to the breathtaking Anse Lazio beach, perfect for snorkeling. The island also offers the chance to see giant Aldabra tortoises on Curieuse Island, combining natural beauty with vibrant marine life.

These destinations provide a perfect mix of relaxation and excitement, ensuring an unforgettable tropical beach vacation.

## Multi-Agent System with `CrewAI`

### `Step 1`: Define a tool to search the web

In [14]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool('DuckDuckGoSearch')
def search_web(search_query: str) -> str:
    """Search the web for information"""
    return DuckDuckGoSearchRun().run(search_query)


### `Step 2`: Define an Agent that uses the tool

### Research Agent and Task

In [15]:
# Research Agent
research_agent = Agent(
    role="Internet Researcher",
    goal="Find the most relevant and recent information about a given topic.",
    backstory="""You are a skilled researcher, adept at navigating the internet 
                 and gathering high-quality, reliable information.""",
    llm=llm,
    verbose=True
)

# Research Task
research_task = Task(
    description="""Use the search_web to search for the 
                   most relevant and recent data about {topic}."""
                "Extract the key insights from multiple sources.",
    agent=research_agent,
    tools=[search_web],
    expected_output="A detailed research report with key insights and source references."
)

### Summarizer Agent and Task

In [16]:
# Summarizer Agent
summarizer_agent = Agent(
    role="Content Summarizer",
    goal="Condense the key insights from research into a short and informative summary.",
    backstory="""You are an expert in distilling complex information into concise, 
                 easy-to-read summaries.""",
    llm=llm,
    verbose=True
)

# Summarization Task
summarization_task = Task(
    description="Summarize the research report into a concise and informative paragraph. "
                "Ensure clarity, coherence, and completeness.",
    agent=summarizer_agent,
    expected_output="A well-structured summary with the most important insights."
)


### Fact-Checking Agent and Task

In [22]:
# Fact-Checking Agent
fact_checker_agent = Agent(
    role="Fact-Checking Specialist",
    goal="Verify the accuracy of information and remove any misleading or false claims.",
    backstory="""You are an investigative journalist with a knack for validating facts, 
                 ensuring that only accurate information is published.""",
    llm=llm,
    verbose=True
)

# Fact-Checking Task
fact_checking_task = Task(
    description="Verify the summarized information for accuracy using the search_web. "
                "Cross-check facts with reliable sources and correct any errors.",
    agent=fact_checker_agent,
    tools=[search_web],
    expected_output="A fact-checked, verified summary of the research topic in a nice markdown format."
)

### Create a Crew with defined Processes


In [27]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)

result = research_crew.kickoff(inputs={"topic": "The impact of DeepSeek on GenAI ecosystem"})
Markdown(result.raw)

2025-02-12 15:42:36,086 - 8232306752 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Internet Researcher
## Task: Use the search_web to search for the 
                   most relevant and recent data about The impact of DeepSeek on GenAI ecosystem.Extract the key insights from multiple sources.


# Agent: Internet Researcher
## Thought: Thought: To gather the most relevant and recent data about the impact of DeepSeek on the GenAI ecosystem, I will perform a series of web searches using DuckDuckGoSearch. I will look for articles, reports, and other resources that provide insights into this topic.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"impact of DeepSeek on GenAI ecosystem\"}"
## Tool Output: 
Chinese start-up DeepSeek rocked the tech industry Monday after the company's new generative AI (genAI) bot hit Apple's App Store and Google Play Store and downloads almost immediately ... As GenAI API costs decline, organizations should reassess AI deployment strategies, balancing cloud-based and on-premises models. Cloud AI offers scalabil

# Research Report Summary on the Impact of DeepSeek on the GenAI Ecosystem

## Introduction
The emergence of DeepSeek, a Chinese start-up, has notably disrupted the tech industry with its cost-effective generative AI (genAI) model, quickly available on major app stores and demonstrating significant innovation.

## Cost-Effectiveness and Innovation
DeepSeek’s model training, achieved with less than $6 million and 2,000 chips, contrasts sharply with the higher resource investments by US tech giants. This efficiency showcases that high-quality AI models can be developed with lower costs, driving innovation within the GenAI ecosystem and lowering barriers to entry for other organizations.

## Impact on AI Deployment Strategies
As GenAI API costs decline, organizations are reassessing their AI deployment strategies, balancing cloud-based and on-premises models. Cloud AI provides scalability and integration benefits, while on-premises solutions offer regulatory compliance and security advantages.

## Economic and Geopolitical Turbulence
The rise of DeepSeek has caused economic and geopolitical turbulence, particularly in the US AI ecosystem, leading to market reactions and raising questions about the competitive landscape of AI.

## Comparisons to the Dotcom Bubble
The rapid growth in the AI sector, driven by companies like DeepSeek, has drawn comparisons to the late 1990s dotcom bubble, highlighting the need for careful evaluation and strategic planning to navigate the evolving GenAI landscape and mitigate associated risks.

## Conclusion
DeepSeek's cost-effective approach and rapid market adoption illustrate a significant trend in AI's evolution, challenging traditional notions of AI development and deployment. As the GenAI ecosystem continues to evolve, the impact of companies like DeepSeek will likely shape the future of AI innovation and competition.

In [29]:
# This won't work as the agent has no memory
result = research_crew.kickoff(inputs={"topic": "When it was released?"})
Markdown(result.raw)

# Agent: Internet Researcher
## Task: Use the search_web to search for the 
                   most relevant and recent data about When it was released?.Extract the key insights from multiple sources.


# Agent: Internet Researcher
## Final Answer: 
Please specify the product or service you want to know the release date of.


# Agent: Content Summarizer
## Task: Summarize the research report into a concise and informative paragraph. Ensure clarity, coherence, and completeness.


# Agent: Content Summarizer
## Final Answer: 
The research report investigates the impact of climate change on coastal ecosystems, focusing on the alteration of biodiversity and the effectiveness of conservation strategies. The study employs a combination of field observations, remote sensing data, and modeling techniques to assess changes over the past three decades. Key findings indicate a significant decline in species diversity and an increase in invasive species, particularly in regions with higher human a

# Research Report Summary on the Impact of Climate Change on Coastal Ecosystems

## Introduction
This research report investigates the impact of climate change on coastal ecosystems, with a focus on the alteration of biodiversity and the effectiveness of conservation strategies. The study utilizes a combination of field observations, remote sensing data, and modeling techniques to assess changes over the past three decades.

## Key Findings

### 1. Impact of Climate Change on Coastal Ecosystems
Climate change has profound effects on coastal ecosystems, leading to changes in temperature, sea level rise, and increased frequency of extreme weather events. These changes directly impact the physical and biological characteristics of coastal habitats.

### 2. Alteration of Biodiversity
The study reveals a significant decline in species diversity in coastal ecosystems. This decline is attributed to various factors, including habitat loss, changes in water temperature, and increased salinity levels.

### 3. Increase in Invasive Species
There is a noticeable increase in invasive species, particularly in regions with higher human activity. Invasive species compete with native species for resources, further threatening biodiversity.

### 4. Effectiveness of Conservation Strategies
The report evaluates the effectiveness of various conservation strategies implemented over the past three decades. While some strategies have shown positive results, others have fallen short due to inadequate funding, lack of community involvement, and insufficient adaptive management practices.

### 5. Use of Field Observations, Remote Sensing Data, and Modeling Techniques
The research employs a comprehensive approach by combining field observations, remote sensing data, and modeling techniques. This multi-method approach provides a robust assessment of changes in coastal ecosystems.

### 6. Importance of Adaptive Management Practices
Adaptive management practices are highlighted as crucial for addressing the dynamic nature of climate change impacts. These practices involve continuous monitoring, evaluation, and adjustment of conservation strategies based on new information and changing conditions.

### 7. Integration of Local Communities in Conservation Efforts
The report underscores the importance of integrating local communities in conservation efforts. Community-based approaches enhance the effectiveness of conservation strategies by leveraging local knowledge and ensuring sustainable resource management.

## Recommendations
To enhance resilience in coastal ecosystems, the report concludes with the following recommendations:
- Implement policy interventions that support adaptive management practices.
- Increase funding for conservation efforts and research.
- Foster collaboration between scientists, policymakers, and local communities.
- Promote sustainable resource management practices.
- Enhance public awareness and education on the impacts of climate change on coastal ecosystems.

## Conclusion
The research report provides valuable insights into the impact of climate change on coastal ecosystems and offers practical recommendations for enhancing resilience through adaptive management and community involvement.

## Agents with `Memory`





In [57]:
# Define agents
researcher = Agent(
    role="Chip Manufacturing Analyst",
    goal="Provide detailed technical analysis",
    backstory="Experienced semiconductor engineer and designer with experience in many leader chip manufacturing companies like TSMC, Samsung, and Intel",
    llm=llm,
    verbose=False
)

analyst = Agent(
    role="Market Intelligence Specialist",
    goal="Identify business implications",
    backstory="Former strategy consultant for fabless semiconductor companies",
    llm=llm,
    verbose=False
)


In [58]:
task1 = Task(
    description="""Research current industry trends related to {user_query}""",
    expected_output="Technical comparison table with key metrics",
    agent=researcher
)

task2 = Task(
    description="""Analyze market impact of {user_query} developments""",
    expected_output="SWOT analysis with investment recommendations", 
    agent=analyst
)

In [59]:
# Configure AWS Bedrock embedding model
bedrock_embedder = {
    "provider": "aws_bedrock",
    "config": {
        "model": "amazon.titan-embed-text-v1",
        "vector_dimension": 1024
    }
}

In [60]:
# Basic crew without memory
basic_crew = Crew(
    agents=[researcher, analyst],
    tasks=[task1, task2],
    process=Process.sequential,
    memory=False,
    verbose=False
)

# Enhanced crew with AWS memory
memory_crew = Crew(
    agents=[researcher, analyst],
    tasks=[task1, task2],
    process=Process.sequential,
    memory=True,
    embedder=bedrock_embedder,
    verbose=False
)

2025-02-12 16:19:31,211 - 8232306752 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2025-02-12 16:19:31,217 - 8232306752 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [62]:
# Execute basic crew
basic_result = basic_crew.kickoff({"user_query": "3nm chip manufacturing yield rates"})
# Execute memory crew
memory_result = memory_crew.kickoff({"user_query": "3nm chip manufacturing yield rates"})

In [63]:
# Display results (basic crew)
Markdown(basic_result.raw)

### SWOT Analysis of 3nm Chip Manufacturing Yield Rates Developments

#### Strengths:
1. **High Yield Rates**:
   - **TSMC**: Yield rates of 70-80% indicate a high level of manufacturing efficiency, which translates to lower production costs and higher profitability.
   - **Samsung**: Yield rates of 65-75% are competitive and show strong manufacturing capabilities.
   - **Intel**: Yield rates of 60-70% demonstrate progress in their 3nm manufacturing process, despite being slightly behind TSMC and Samsung.

2. **Advanced Process Technologies**:
   - All companies are leveraging cutting-edge process technologies (N3, 3GAE, Intel 3) that promise significant improvements in PPA metrics.

3. **EUV Lithography Usage**:
   - **TSMC** and **Samsung** have high and extensive EUV usage, respectively, which is critical for achieving fine patterns at the 3nm node.
   - **Intel’s** moderate EUV usage indicates a balanced approach, though it may lag slightly in process maturity compared to TSMC and Samsung.

4. **Defect Density**:
   - Low defect densities across all companies (below 1500 defects/cm²) signify mature manufacturing processes, which is crucial for high yield rates.

5. **PPA Improvements**:
   - **TSMC** leads with significant PPA gains, followed by **Samsung** with notable improvements, and **Intel** with moderate gains. These improvements are vital for enhancing chip performance and reducing power consumption.

#### Weaknesses:
1. **Lower Yield Rates for Intel**:
   - Intel’s yield rates (60-70%) are the lowest among the three, indicating potential challenges in process maturity and efficiency.

2. **Moderate EUV Usage by Intel**:
   - Intel’s moderate EUV usage may limit its ability to achieve the same level of process precision as TSMC and Samsung.

3. **Defect Density**:
   - Although all companies have low defect densities, Intel’s higher defect density (below 1500 defects/cm²) compared to TSMC (below 1000 defects/cm²) and Samsung (below 1200 defects/cm²) could impact yield rates and production efficiency.

#### Opportunities:
1. **Market Leadership**:
   - TSMC’s early entry into volume production (late 2022 - early 2023) positions it well to capture a significant market share.
   - Samsung and Intel can still capitalize on their expected volume production timelines (mid 2023 for Samsung and late 2023 - early 2024 for Intel) to gain market share.

2. **Technological Advancements**:
   - Continuous improvements in EUV technology and process refinements can lead to higher yield rates and better PPA metrics for all companies.

3. **Geopolitical Advantages**:
   - TSMC’s manufacturing in Taiwan, Samsung’s in South Korea, and Intel’s in the USA provide geopolitical stability and reduce supply chain risks.

#### Threats:
1. **Competition**:
   - Intense competition among TSMC, Samsung, and Intel could lead to price wars and margin compression.
   
2. **Geopolitical Risks**:
   - Trade tensions and geopolitical instability in Taiwan, South Korea, and the USA could disrupt manufacturing operations and supply chains.

3. **Technological Challenges**:
   - Scaling down to 3nm presents significant technological challenges, including higher defect densities and lower yield rates, which could impact production efficiency and costs.

### Investment Recommendations:

1. **TSMC**:
   - **Buy**: TSMC’s high yield rates, advanced process technology, and early volume production make it a strong investment candidate. Its leadership in EUV usage and low defect density further enhance its competitive position.

2. **Samsung**:
   - **Hold**: Samsung’s competitive yield rates and extensive EUV usage position it well in the market. However, its slightly later expected volume production (mid 2023) compared to TSMC may warrant a hold recommendation until it demonstrates stronger market performance.

3. **Intel**:
   - **Wait and Watch**: Intel’s lower yield rates and moderate EUV usage present risks. Investors should wait for further improvements in its 3nm process before considering an investment. Its expected volume production timeline (late 2023 - early 2024) also suggests a need for caution.

In [64]:
# Display results (memory crew)
Markdown(memory_result.raw)

### SWOT Analysis of 3nm Chip Manufacturing Yield Rates Developments

#### Strengths

1. **TSMC**:
   - **High Yield Rate**: TSMC’s 3nm process boasts a yield rate of approximately 70-80%, reflecting their leadership in yield optimization.
   - **Early Market Entry**: Started mass production in Q4 2022, giving them a competitive edge.
   - **Advanced Technologies**: Utilization of Gate-All-Around (GAA) FETs, EUV lithography, and cobalt-based metal layers enhances performance and reduces power consumption.

2. **Samsung**:
   - **Innovative Technologies**: Adoption of Spacer FET technology, EUV lithography, and advanced packaging techniques positions Samsung strongly in the market.
   - **Early Production Start**: Began 3nm production in Q2 2022, allowing them to capture early market share.

3. **Intel**:
   - **Innovative Approaches**: Introduction of RibbonFET and PowerVia technologies aims to improve yield and performance.
   - **EUV Lithography**: Leveraging EUV lithography to achieve better scaling and efficiency.

#### Weaknesses

1. **TSMC**:
   - **Variability Management**: Challenges in managing variability at smaller nodes.
   - **Thermal Management**: Critical concerns around thermal management in smaller nodes.

2. **Samsung**:
   - **Yield Optimization**: Struggles with yield optimization and reducing defect density.
   - **Complex Processes**: The complexity of their manufacturing processes adds to the challenges.

3. **Intel**:
   - **Yield Ramp-Up**: Faces significant yield ramp-up challenges.
   - **Catch-Up Mode**: Playing catch-up with competitors who have already established 3nm processes.
   - **Process Integration**: Complexities in process integration pose additional hurdles.

#### Opportunities

1. **Market Leadership**: Companies with higher yield rates (like TSMC) can capture a larger market share and set industry standards.
2. **Technological Advancements**: Continued innovation in FET technologies, EUV lithography, and packaging can lead to further yield improvements and performance gains.
3. **Growing Demand**: The increasing demand for high-performance chips in data centers, AI, and automotive sectors presents significant growth opportunities.

#### Threats

1. **Competition**: Intense competition among leading foundries can lead to price wars and margin pressures.
2. **Yield Variability**: Inconsistent yield rates can lead to production bottlenecks and increased costs.
3. **Geopolitical Factors**: Trade tensions and geopolitical factors can impact supply chains and market access.

### Investment Recommendations

1. **TSMC**:
   - **Recommendation**: Strong Buy
   - **Rationale**: TSMC’s leading yield rates, early market entry, and advanced technologies position them as a top investment choice in the semiconductor foundry space.

2. **Samsung**:
   - **Recommendation**: Buy
   - **Rationale**: Samsung’s innovative technologies and early production start make them a strong contender. However, challenges in yield optimization should be monitored.

3. **Intel**:
   - **Recommendation**: Hold
   - **Rationale**: Intel’s innovative approaches and planned 3nm production in 2023 show promise. However, their yield ramp-up challenges and need to catch up with competitors warrant a cautious approach.